# Sine Model

An Sine analysis and synthesis notebook.

First we set up the environment.

In [1]:
%matplotlib inline

import math, copy, sys, os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import IPython.display as ipd
import glob
from scipy.fftpack import fft, ifft, fftshift
from scipy.signal import blackmanharris, triang, get_window
from scipy.io.wavfile import write, read
from sys import platform
from ipywidgets import interact, interact_manual, interactive

tol = 1e-14 # threshold used to compute phase
INT16_FAC = (2**15)-1
INT32_FAC = (2**31)-1
INT64_FAC = (2**63)-1
norm_fact = {'int16':INT16_FAC, 'int32':INT32_FAC, 'int64':INT64_FAC,'float32':1.0,'float64':1.0}

global iF  # The input file name
global xR  # The raw input samples 
global x   # The input samples normalized
global fs  # The input sample rate
global N   # The FFT size
global w   # The window
global wN  # The window name
global M   # The window size
global H   # The hop size
global mX  # The magnitude spectrum of the input
global pX  # The phase spectrum of the input
global y   # The re-synthesized output
global yR  # The raw re-synthesized output

Now we define some methods to perform the different steps of the model

***dft_analysis***

Analysis of a signal using the discrete Fourier transform 

Params

* x: input signal 
* w: analysis window, 
* N: FFT size 

Returns 

* mX: magnitude spectrum
* pX: phase spectrum

In [2]:
def dft_analysis(x, w, N):
    if (w.size > N):                                        # raise error if window size bigger than fft size
        raise ValueError("Window size (M) is bigger than FFT size")

    hN = (N//2)+1                                           # size of positive spectrum, it includes sample 0
    hM1 = (w.size+1)//2                                     # half analysis window size by rounding
    hM2 = w.size//2                                         # half analysis window size by floor
    fftbuffer = np.zeros(N)                                 # initialize buffer for FFT
    w = w / sum(w)                                          # normalize analysis window
    xw = x*w                                                # window the input sound
    fftbuffer[:hM1] = xw[hM2:]                              # zero-phase window in fftbuffer
    fftbuffer[-hM2:] = xw[:hM2]        
    X = fft(fftbuffer)                                      # compute FFT
    absX = abs(X[:hN])                                      # compute ansolute value of positive side
    absX[absX<np.finfo(float).eps] = np.finfo(float).eps    # if zeros add epsilon to handle log
    mX = 20 * np.log10(absX)                                # magnitude spectrum of positive frequencies in dB
    X[:hN].real[np.abs(X[:hN].real) < tol] = 0.0            # for phase calculation set to 0 the small values
    X[:hN].imag[np.abs(X[:hN].imag) < tol] = 0.0            # for phase calculation set to 0 the small values         
    pX = np.unwrap(np.angle(X[:hN]))                        # unwrapped phase spectrum of positive frequencies
    return mX, pX

***stft_analysis*** 

Analysis of a sound using the short-time Fourier transform

Params

* x: input array sound
* w: analysis window
* N: FFT size
* H: hop size

Returns 

* xmX: magnitude spectra
* xpX: phase spectra

In [3]:
def stft_analysis(x, w, N, H) :
    if (H <= 0):                                   # raise error if hop size 0 or negative
        raise ValueError("Hop size (H) smaller or equal to 0")

    M = w.size                                      # size of analysis window
    hM1 = (M+1)//2                                  # half analysis window size by rounding
    hM2 = M//2                                      # half analysis window size by floor
    x = np.append(np.zeros(hM2),x)                  # add zeros at beginning to center first window at sample 0
    x = np.append(x,np.zeros(hM2))                  # add zeros at the end to analyze last sample
    pin = hM1                                       # initialize sound pointer in middle of analysis window       
    pend = x.size-hM1                               # last sample to start a frame
    w = w / sum(w)                                  # normalize analysis window
    xmX = []                                       # Initialise empty list for mX
    xpX = []                                       # Initialise empty list for pX
    while pin<=pend:                                # while sound pointer is smaller than last sample      
        x1 = x[pin-hM1:pin+hM2]                     # select one frame of input sound
        mX, pX = dft_analysis(x1, w, N)             # compute dft
        xmX.append(np.array(mX))                    # Append output to list
        xpX.append(np.array(pX))
        pin += H                                    # advance sound pointer
    xmX = np.array(xmX)                             # Convert to numpy array
    xpX = np.array(xpX)
    return xmX, xpX

***dft_synthesis***

Synthesis of a signal using the discrete Fourier transform

Params

* mX: magnitude spectrum 
* pX: phase spectrum
* M: window size

Returns 

* y: output signal

In [4]:
def dft_synthesis(mX, pX, M):
    hN = mX.size                                            # size of positive spectrum, it includes sample 0
    N = (hN-1)*2                                            # FFT size
    hM1 = int(math.floor((M+1)/2))                          # half analysis window size by rounding
    hM2 = int(math.floor(M/2))                              # half analysis window size by floor
    fftbuffer = np.zeros(N)                                 # initialize buffer for FFT
    y = np.zeros(M)                                         # initialize output array
    Y = np.zeros(N, dtype = complex)                        # clean output spectrum
    Y[:hN] = 10**(mX/20) * np.exp(1j*pX)                    # generate positive frequencies
    Y[hN:] = 10**(mX[-2:0:-1]/20) * np.exp(-1j*pX[-2:0:-1]) # generate negative frequencies
    fftbuffer = np.real(ifft(Y))                            # compute inverse FFT
    y[:hM2] = fftbuffer[-hM2:]                              # undo zero-phase window
    y[hM2:] = fftbuffer[:hM1]
    return y

***stft_synthesis***

Synthesis of a sound using the short-time Fourier transform

* mY: magnitude spectra
* pY: phase spectra
* M: window size 
* H: hop-size

Returns 

* y: output sound

In [5]:
def stft_synthesis(mY, pY, M, H) :
    hM1 = (M+1)//2                                   # half analysis window size by rounding
    hM2 = M//2                                       # half analysis window size by floor
    nFrames = mY[:,0].size                           # number of frames
    y = np.zeros(nFrames*H + hM1 + hM2)              # initialize output array
    pin = hM1                  
    for i in range(nFrames):                         # iterate over all frames      
        y1 = dft_synthesis(mY[i,:], pY[i,:], M)           # compute idft
        y[pin-hM1:pin+hM2] += H*y1                   # overlap-add to generate output sound
        pin += H                                     # advance sound pointer
    y = np.delete(y, range(hM2))                     # delete half of first window which was added in stftAnal
    y = np.delete(y, range(y.size-hM1, y.size))      # delete the end of the sound that was added in stftAnal
    return y

***peak_detection***

Detect spectral peak locations

Params

* mX: magnitude spectrum
* t: threshold

Returns 

* ploc: peak locations

In [6]:
def peak_detection(mX, t):
    thresh = np.where(np.greater(mX[1:-1],t), mX[1:-1], 0)  # locations above threshold
    next_minor = np.where(mX[1:-1]>mX[2:], mX[1:-1], 0)     # locations higher than the next one
    prev_minor = np.where(mX[1:-1]>mX[:-2], mX[1:-1], 0)    # locations higher than the previous one
    ploc = thresh * next_minor * prev_minor                 # locations fulfilling the three criteria
    ploc = ploc.nonzero()[0] + 1                            # add 1 to compensate for previous steps
    return ploc

***peak_interpolation***

Interpolate peak values using parabolic interpolation

Params

* mX: magnitude spectrum
* pX: phase spectrum
* ploc: locations of peaks

Returns 

* iploc: interpolated peak location values
* ipmag: interpolated peak location magnitude
* ipphase: interpolated peak phase values

In [7]:
def peak_interpolation(mX, pX, ploc):
    val = mX[ploc]                                          # magnitude of peak bin
    lval = mX[ploc-1]                                       # magnitude of bin at left
    rval = mX[ploc+1]                                       # magnitude of bin at right
    iploc = ploc + 0.5*(lval-rval)/(lval-2*val+rval)        # center of parabola
    ipmag = val - 0.25*(lval-rval)*(iploc-ploc)             # magnitude of peaks
    ipphase = np.interp(iploc, np.arange(0, pX.size), pX)   # phase of peaks by linear interpolation
    return iploc, ipmag, ipphase

***sine_tracking***

Tracking sinusoids from one frame to the next

Params

* pfreq: frequencies of the current frame
* pmag: magnitudes of the current frame
* pphase: phasen of current frame
* tfreq: frequencies of incoming tracks from previous frame
* freqDevOffset: minimum frequency deviation at 0Hz 
* freqDevSlope: slope increase of minimum frequency deviation

Returns 

* tfreqn: frequency of the tracks
* tmagn: magnitude of the tracks
* tphasen: phase of tracks

In [8]:
def sine_tracking(pfreq, pmag, pphase, tfreq, freqDevOffset=20, freqDevSlope=0.01):
    tfreqn = np.zeros(tfreq.size)                              # initialize array for output frequencies
    tmagn = np.zeros(tfreq.size)                               # initialize array for output magnitudes
    tphasen = np.zeros(tfreq.size)                             # initialize array for output phases
    pindexes = np.array(np.nonzero(pfreq), dtype=np.int)[0]    # indexes of current peaks
    incomingTracks = np.array(np.nonzero(tfreq), dtype=np.int)[0] # indexes of incoming tracks
    newTracks = np.zeros(tfreq.size, dtype=np.int) -1           # initialize to -1 new tracks
    magOrder = np.argsort(-pmag[pindexes])                      # order current peaks by magnitude
    pfreqt = np.copy(pfreq)                                     # copy current peaks to temporary array
    pmagt = np.copy(pmag)                                       # copy current peaks to temporary array
    pphaset = np.copy(pphase)                                   # copy current peaks to temporary array

    # continue incoming tracks
    if incomingTracks.size > 0:                                 # if incoming tracks exist
        for i in magOrder:                                        # iterate over current peaks
            if incomingTracks.size == 0:                            # break when no more incoming tracks
                break
            track = np.argmin(abs(pfreqt[i] - tfreq[incomingTracks]))   # closest incoming track to peak
            freqDistance = abs(pfreq[i] - tfreq[incomingTracks[track]]) # measure freq distance
            if freqDistance < (freqDevOffset + freqDevSlope * pfreq[i]):  # choose track if distance is small 
                    newTracks[incomingTracks[track]] = i                      # assign peak index to track index
                    incomingTracks = np.delete(incomingTracks, track)         # delete index of track in incomming tracks
    indext = np.array(np.nonzero(newTracks != -1), dtype=np.int)[0]   # indexes of assigned tracks
    if indext.size > 0:
        indexp = newTracks[indext]                                    # indexes of assigned peaks
        tfreqn[indext] = pfreqt[indexp]                               # output freq tracks 
        tmagn[indext] = pmagt[indexp]                                 # output mag tracks 
        tphasen[indext] = pphaset[indexp]                             # output phase tracks 
        pfreqt= np.delete(pfreqt, indexp)                             # delete used peaks
        pmagt= np.delete(pmagt, indexp)                               # delete used peaks
        pphaset= np.delete(pphaset, indexp)                           # delete used peaks

    # create new tracks from non used peaks
    emptyt = np.array(np.nonzero(tfreq == 0), dtype=np.int)[0]      # indexes of empty incoming tracks
    peaksleft = np.argsort(-pmagt)                                  # sort left peaks by magnitude
    if ((peaksleft.size > 0) & (emptyt.size >= peaksleft.size)):    # fill empty tracks
            tfreqn[emptyt[:peaksleft.size]] = pfreqt[peaksleft]
            tmagn[emptyt[:peaksleft.size]] = pmagt[peaksleft]
            tphasen[emptyt[:peaksleft.size]] = pphaset[peaksleft]
    elif ((peaksleft.size > 0) & (emptyt.size < peaksleft.size)):   # add more tracks if necessary
            tfreqn[emptyt] = pfreqt[peaksleft[:emptyt.size]]
            tmagn[emptyt] = pmagt[peaksleft[:emptyt.size]]
            tphasen[emptyt] = pphaset[peaksleft[:emptyt.size]]
            tfreqn = np.append(tfreqn, pfreqt[peaksleft[emptyt.size:]])
            tmagn = np.append(tmagn, pmagt[peaksleft[emptyt.size:]])
            tphasen = np.append(tphasen, pphaset[peaksleft[emptyt.size:]])
    return tfreqn, tmagn, tphasen

***sine_model_analysis***
 
Analysis of a sound using the sinusoidal model with sine tracking
 
Params

* x: input array sound
* w: analysis window
* N: size of complex spectrum 
* H: hop-size 
* t: threshold in negative dB
* maxnSines: maximum number of sines per frame, minSineDur: minimum duration of sines in seconds
* freqDevOffset: minimum frequency deviation at 0Hz, 
* freqDevSlope: slope increase of minimum frequency deviation

Returns 

* xtfreq: frequencies of sinusoidal tracks
* xtmag: magnitudes of sinusoidal tracks
* xtphase: phases of sinusoidal tracks

In [9]:
def sine_model_analysis(x, fs, w, N, H, t, maxnSines = 100, freqDevOffset=20, freqDevSlope=0.01):
    hM1 = int(math.floor((w.size+1)/2))                     # half analysis window size by rounding
    hM2 = int(math.floor(w.size/2))                         # half analysis window size by floor
    x = np.append(np.zeros(hM2),x)                          # add zeros at beginning to center first window at sample 0
    x = np.append(x,np.zeros(hM2))                          # add zeros at the end to analyze last sample
    pin = hM1                                               # initialize sound pointer in middle of analysis window       
    pend = x.size - hM1                                     # last sample to start a frame
    w = w / sum(w)                                          # normalize analysis window
    tfreq = np.array([])
    while pin<pend:                                         # while input sound pointer is within sound            
        x1 = x[pin-hM1:pin+hM2]                               # select frame
        mX, pX = dft_analysis(x1, w, N)                       # compute dft
        ploc = peak_detection(mX, t)                          # detect locations of peaks
        iploc, ipmag, ipphase = peak_interpolation(mX, pX, ploc)# refine peak values by interpolation
        ipfreq = fs*iploc/float(N)                            # convert peak locations to Hertz
        # perform sinusoidal tracking by adding peaks to trajectories
        tfreq, tmag, tphase = sine_tracking(ipfreq, ipmag, ipphase, tfreq, freqDevOffset, freqDevSlope)
        tfreq = np.resize(tfreq, min(maxnSines, tfreq.size))  # limit number of tracks to maxnSines
        tmag = np.resize(tmag, min(maxnSines, tmag.size))     # limit number of tracks to maxnSines
        tphase = np.resize(tphase, min(maxnSines, tphase.size)) # limit number of tracks to maxnSines
        jtfreq = np.zeros(maxnSines)                          # temporary output array
        jtmag = np.zeros(maxnSines)                           # temporary output array
        jtphase = np.zeros(maxnSines)                         # temporary output array   
        jtfreq[:tfreq.size]=tfreq                             # save track frequencies to temporary array
        jtmag[:tmag.size]=tmag                                # save track magnitudes to temporary array
        jtphase[:tphase.size]=tphase                          # save track magnitudes to temporary array
        if pin == hM1:                                        # if first frame initialize output sine tracks
            xtfreq = jtfreq 
            xtmag = jtmag
            xtphase = jtphase
        else:                                                 # rest of frames append values to sine tracks
            xtfreq = np.vstack((xtfreq, jtfreq))
            xtmag = np.vstack((xtmag, jtmag))
            xtphase = np.vstack((xtphase, jtphase))
        pin += H
    return xtfreq, xtmag, xtphase

***generate_spectrum_from_sines***

Generate a spectrum from a series of sine values

Params

* iploc: sine peaks locations
* ipmag: sine peaks magnitudes
* ipphase: sine peaks phases
* N: size of the complex spectrum to generate 
* fs: sampling rate

Returns 

* Y: generated complex spectrum of sines

In [10]:
def generate_spectrum_from_sines(ipfreq, ipmag, ipphase, N, fs):
    def sinc(x, N):
        """
        Generate the main lobe of a sinc function (Dirichlet kernel)
        x: array of indexes to compute; N: size of FFT to simulate
        returns y: samples of the main lobe of a sinc function
        """

        y = np.sin(N * x/2) / np.sin(x/2)                  # compute the sinc function
        y[np.isnan(y)] = N                                 # avoid NaN if x == 0
        return y
    
    def generate_bh_lobe(x):
        """
        Generate the main lobe of a Blackman-Harris window
        x: bin positions to compute (real values)
        returns y: main lobe os spectrum of a Blackman-Harris window
        """

        N = 512                                                 # size of fft to use
        f = x*np.pi*2/N                                         # frequency sampling
        df = 2*np.pi/N
        y = np.zeros(x.size)                                    # initialize window
        consts = [0.35875, 0.48829, 0.14128, 0.01168]           # window constants
        for m in range(0,4):                                    # iterate over the four sincs to sum
            y += consts[m]/2 * (sinc(f-df*m, N) + sinc(f+df*m, N))  # sum of scaled sinc functions
        y = y/N/consts[0]                                       # normalize
        return y
    
    Y = np.zeros(N, dtype = complex)                 # initialize output complex spectrum
    hN = N//2                                        # size of positive freq. spectrum
    for i in range(0, ipfreq.size):                  # generate all sine spectral lobes
        loc = N * ipfreq[i] / fs                       # it should be in range ]0,hN-1[
        if loc==0 or loc>hN-1: continue
        binremainder = round(loc)-loc
        lb = np.arange(binremainder-4, binremainder+5) # main lobe (real value) bins to read
        lmag = generate_bh_lobe(lb) * 10**(ipmag[i]/20)       # lobe magnitudes of the complex exponential
        b = np.arange(round(loc)-4, round(loc)+5)
        for m in range(0, 9):
            if b[m] < 0:                                 # peak lobe crosses DC bin
                Y[-int(b[m])] += lmag[m]*np.exp(-1j*ipphase[i])
            elif b[m] > hN:                              # peak lobe croses Nyquist bin
                Y[int(b[m])] += lmag[m]*np.exp(-1j*ipphase[i])
            elif b[m] == 0 or b[m] == hN:                # peak lobe in the limits of the spectrum
                Y[int(b[m])] += lmag[m]*np.exp(1j*ipphase[i]) + lmag[m]*np.exp(-1j*ipphase[i])
            else:                                        # peak lobe in positive freq. range
                Y[int(b[m])] += lmag[m]*np.exp(1j*ipphase[i])
        Y[hN+1:] = Y[hN-1:0:-1].conjugate()            # fill the negative part of the spectrum
    return Y

***sine_model_synthesis***

Synthesis of a sound using the sinusoidal model

* tfreq: frequencies of sinusoids
* tmag: magnitudes of sinusoids
* tphase: phases of sinusoids
* N: synthesis FFT size 
* H: hop size
* fs: sampling rate

Returns 

* y: output array sound

In [11]:
def sine_model_synthesis(tfreq, tmag, tphase, N, H, fs):
    hN = N//2                                               # half of FFT size for synthesis
    L = tfreq.shape[0]                                      # number of frames
    pout = 0                                                # initialize output sound pointer         
    ysize = H*(L+3)                                         # output sound size
    y = np.zeros(ysize)                                     # initialize output array
    sw = np.zeros(N)                                        # initialize synthesis window
    ow = triang(2*H)                                        # triangular window
    sw[hN-H:hN+H] = ow                                      # add triangular window
    bh = blackmanharris(N)                                  # blackmanharris window
    bh = bh / sum(bh)                                       # normalized blackmanharris window
    sw[hN-H:hN+H] = sw[hN-H:hN+H]/bh[hN-H:hN+H]             # normalized synthesis window
    lastytfreq = tfreq[0,:]                                 # initialize synthesis frequencies
    ytphase = 2*np.pi*np.random.rand(tfreq[0,:].size)       # initialize synthesis phases 
    for l in range(L):                                      # iterate over all frames
        if (tphase.size > 0):                                 # if no phases generate them
            ytphase = tphase[l,:] 
        else:
            ytphase += (np.pi*(lastytfreq+tfreq[l,:])/fs)*H     # propagate phases
        Y = generate_spectrum_from_sines(tfreq[l,:], tmag[l,:], ytphase, N, fs)  # generate sines in the spectrum         
        lastytfreq = tfreq[l,:]                               # save frequency for phase propagation
        ytphase = ytphase % (2*np.pi)                         # make phase inside 2*pi
        yw = np.real(fftshift(ifft(Y)))                       # compute inverse FFT
        y[pout:pout+N] += sw*yw                               # overlap-add and apply a synthesis window
        pout += H                                             # advance sound pointer
    y = np.delete(y, range(hN))                             # delete half of first window
    y = np.delete(y, range(y.size-hN, y.size))              # delete half of the last window 
    return y

***sine_model_system***

Sine model analysis and re-synthesis system. Performs an sine model analysis of a signal and then re-synthesizes it

Params

* p_N:  The FFT size
* p_M:  The window size
* p_H:  The hop size
* p_wN: The name of the window funtion to use
* p_t: The magnitude threshold over which to track sines
* p_maxnSines: the max number of sines to track

Returns void

Plots the input waveform, the magnitude and phase spectra, and the re-synthesized output waveform and allows the output to be played back


In [12]:
def sine_model_system(p_N, p_M, p_H, p_wN, p_t, p_maxnSines):
    global N, M, H, wN, w, mX, pX, y, yR
    
    # Set the analysis parameters
    N = p_N
    M = p_M if p_M <= N else N
    H = p_H if p_H <= M//2 else M//2
    wN = p_wN
    w = get_window(wN, M)
    
    t=p_t
    maxnSines=p_maxnSines
    freqDevOffset=10
    freqDevSlope=0.001

    # analyze the sound with the sinusoidal model
    tfreq, tmag, tphase = sine_model_analysis(x, fs, w, N, H, t, maxnSines, freqDevOffset, freqDevSlope)

    # size of fft used in synthesis
    Ns = 512
    # hop size (has to be 1/4 of Ns)
    Hs = 128
    # synthesize the output sound from the sinusoidal representation
    y = sine_model_synthesis(tfreq, tmag, tphase, Ns, Hs, fs)
    
    yR = copy.deepcopy(y)                         # copy array
    yR *= INT16_FAC                               # scaling floating point -1 to 1 range signal to int16 range
    yR = np.int16(yR)

    # create figure to show plots
    plt.figure(figsize=(12, 9))

    # frequency range to plot
    maxplotfreq = 5000.0

    # plot the input sound
    plt.subplot(3,1,1)
    plt.plot(np.arange(x.size)/float(fs), x)
    plt.axis([0, x.size/float(fs), min(x), max(x)])
    plt.ylabel('amplitude')
    plt.xlabel('time (sec)')
    plt.title('input sound: x')

    # plot the sinusoidal frequencies
    plt.subplot(3,1,2)
    if (tfreq.shape[1] > 0):
        numFrames = tfreq.shape[0]
        frmTime = H*np.arange(numFrames)/float(fs)
        tfreq[tfreq<=0] = np.nan
        plt.plot(frmTime, tfreq)
        plt.axis([0, x.size/float(fs), 0, maxplotfreq])
        plt.title('frequencies of sinusoidal tracks')

    # plot the output sound
    plt.subplot(3,1,3)
    plt.plot(np.arange(y.size)/float(fs), y)
    plt.axis([0, y.size/float(fs), min(y), max(y)])
    plt.ylabel('amplitude')
    plt.xlabel('time (sec)')
    plt.title('output sound: y')

    plt.tight_layout()
    plt.ion()
    plt.show()
    
    display(ipd.Audio(yR, rate=fs))

# Playground

Here you can play with a few different inputs, change some parameters and listen to the results

In [13]:
def read_input_file(p_iF):
    global iF, fs, xR, x
    iF = p_iF
    # Read the input file now
    fs, xR = read(iF)
    x = np.float32(xR)/norm_fact[xR.dtype.name]
    display(ipd.Audio(xR, rate=fs))
    
files = glob.glob('audio/*.wav')
interact(read_input_file, p_iF = widgets.Dropdown(options=files,description='Audio File:'))
interact_manual(sine_model_system,
         p_wN = widgets.Dropdown(options=['blackmanharris', 'blackman', 'hamming', 'hanning', 'rectangular' ],description='Window Type'),
         p_M=widgets.SelectionSlider(options=[2**i for i in range(4,13)],value=512,description='Window Size'),
         p_N=widgets.SelectionSlider(options=[2**i for i in range(4,13)],value=1024,description='FFT Size'), 
         p_H=widgets.SelectionSlider(options=[2**i for i in range(4,13)],value=128,description='Hop Size'),
         p_t=widgets.IntSlider(value=-80,min=-120,max=-20,step=5,description='Mag. Threshold'),
         p_maxnSines=widgets.IntSlider(value=16,min=1,max=100,step=1,description='Max. Sine Tracks')
) 

interactive(children=(Dropdown(description='Audio File:', options=('audio/speech-female.wav', 'audio/sine-440.wav', 'audio/speech-male.wav', 'audio/singing-female.wav', 'audio/sine-490.wav', 'audio/sine-440-490.wav', 'audio/cello-phrase.wav', 'audio/oboe-A4.wav', 'audio/sax-phrase-short.wav'), value='audio/speech-female.wav'), Output()), _dom_classes=('widget-interact',))

interactive(children=(SelectionSlider(description='FFT Size', index=6, options=(16, 32, 64, 128, 256, 512, 1024, 2048, 4096), value=1024), SelectionSlider(description='Window Size', index=5, options=(16, 32, 64, 128, 256, 512, 1024, 2048, 4096), value=512), SelectionSlider(description='Hop Size', index=3, options=(16, 32, 64, 128, 256, 512, 1024, 2048, 4096), value=128), Dropdown(description='Window Type', options=('blackmanharris', 'blackman', 'hamming', 'hanning', 'rectangular'), value='blackmanharris'), IntSlider(value=-80, description='Mag. Threshold', max=-20, min=-120, step=5), IntSlider(value=16, description='Max. Sine Tracks', min=1), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.sine_model_system>